# An oversimplified energy system model

In [1]:
import pyomo.environ as pyo
import pandas as pd
import plotly.express as px

## Function definitions

In [2]:
def model_vars_as_dfs(model, debug=False) -> dict[str, pd.DataFrame]:
    df_dict = {}
    # loop over all variables
    for m_var in model.component_objects(pyo.Var, active=True):
        # store values in a series
        var_values = pd.Series(m_var.get_values())

        # get names of indices
        subsets = list(m_var.index_set().subsets())
        subset_names = [x.name for x in subsets]

        # print information if desired
        if debug:
            print(f"{m_var} is indexed in {subset_names}")

        # transform to dataframe and store in dictionary
        df = var_values.to_frame().reset_index()
        df.columns = subset_names + ["value"]
        df_dict[f"{m_var}"] = df.copy()

    return df_dict

In [3]:
def get_shadow_prices(model, balance_constraint_name="balance_constraint"):
    constraints = []
    for c in esm.component_objects(pyo.Constraint, active=True):
        # export only the marginals of constraints in that list
        constraints.append(c)
        vals = {}
        if str(c) == balance_constraint_name:
            for index in c:
                vals[index] = esm.dual[c[index]]
        dual_df = pd.DataFrame(vals.values(), index=vals.keys(), columns=["dual"])
        return dual_df
    
    print(balance_constraint_name, " not found. Model constains these constraints:", constraints)

## gather time series

In [4]:
from pathlib import Path

solar_data_path = "solar_data.csv"
if not Path(solar_data_path).exists():
    # Toronto:
    lat, lon =43.712148, -79.304524
    pv_peak_power = 1
    url = f"https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={lat}&lon={lon}&raddatabase=PVGIS-NSRDB&browser=1&outputformat=csv&userhorizon=&usehorizon=1&angle=&aspect=&startyear=2014&endyear=2014&mountingplace=free&optimalinclination=0&optimalangles=1&js=1&select_database_hourly=PVGIS-NSRDB&hstartyear=2014&hendyear=2014&trackingtype=0&hourlyoptimalangles=1&pvcalculation=1&pvtechchoice=crystSi&peakpower={pv_peak_power}&loss=0&components=1"
    solar_data = pd.read_csv(url, header=8)

    # clear values that are not available
    na = solar_data["T2m"].isna()
    solar_data = solar_data.loc[~na,:]
    solar_data.loc[:,"P":] = solar_data.loc[:,"P":].astype(float)
    solar_data["cf"] = solar_data["P"] / (pv_peak_power*1000)
    solar_data.to_csv(solar_data_path)
else:
    solar_data = pd.read_csv(solar_data_path, index_col=0)
solar_data.head()


,time,P,Gb(i),Gd(i),Gr(i),H_sun,T2m,WS10m,Int,cf
0,20140101:0000,0.0,0.0,0.0,0.0,0.0,-9.60,7.52,0.0,0.0
1,20140101:0100,0.0,0.0,0.0,0.0,0.0,-9.73,6.90,0.0,0.0
2,20140101:0200,0.0,0.0,0.0,0.0,0.0,-9.70,6.41,0.0,0.0
3,20140101:0300,0.0,0.0,0.0,0.0,0.0,-9.61,6.62,0.0,0.0
4,20140101:0400,0.0,0.0,0.0,0.0,0.0,-9.53,6.62,0.0,0.0


In [5]:
eff_df = solar_data["cf"].to_frame()
eff_df.loc[:,"coal"] = 0.38
eff_df.loc[:,"gas"] = 0.5
eff_df.rename({"cf":"pv"}, axis=1, inplace=True)
eff_df


,pv,coal,gas
0,0.00000,0.38,0.5
1,0.00000,0.38,0.5
2,0.00000,0.38,0.5
3,0.00000,0.38,0.5
4,0.00000,0.38,0.5
...,...,...,...
8755,0.30922,0.38,0.5
8756,0.06268,0.38,0.5
8757,0.04581,0.38,0.5
8758,0.00000,0.38,0.5


In [6]:
eff_df_long = eff_df.melt(ignore_index=False).reset_index().set_index(["variable","index"])
eff_df_long = eff_df_long.sort_index()
eff_df_long.head()

value
variable index       
coal     0       0.38
         1       0.38
         2       0.38
         3       0.38
         4       0.38

In [7]:
ontario_demand_demand_path = "ontario_demand.csv"

if not Path(ontario_demand_demand_path).exists():
    ontario_demand = pd.read_csv("http://reports.ieso.ca/public/Demand/PUB_Demand_2014.csv", header=3)
    ontario_demand.to_csv(ontario_demand_demand_path)
else:
    ontario_demand = pd.read_csv(ontario_demand_demand_path, index_col=0)

# ontario_demand["Ontario Demand"] *= 1/ontario_demand["Ontario Demand"].sum()
ontario_demand.tail()


,Date,Hour,Market Demand,Ontario Demand
8755,2014-12-31,20,21226,18010
8756,2014-12-31,21,20404,17294
8757,2014-12-31,22,19710,16759
8758,2014-12-31,23,19167,16123
8759,2014-12-31,24,18614,15469


In [8]:
import numpy as np
random_demand = np.random.random(len(ontario_demand["Ontario Demand"])) * max(ontario_demand["Ontario Demand"])
# ontario_demand["Ontario Demand"] += random_demand

# Model definition

In [9]:
def define_model(demand_response=False, demand_ts:pd.Series=ontario_demand["Ontario Demand"], t_start=0, t_end=8759):

    esm = pyo.ConcreteModel("Oversimplified ontario electricity model")

    esm.hour = pyo.RangeSet(t_start, t_end)
    esm.gen_types = pyo.Set(initialize=["pv", "coal", "gas"])

    esm.demand = pyo.Param(esm.hour, initialize=demand_ts.loc[t_start:t_end].to_dict(), within=pyo.NonNegativeReals)
    esm.fuel_cost = pyo.Param(esm.gen_types, initialize={"pv":1e-3, "coal":207, "gas":309}) # add 1 cost to pv, to avoid oversupply
    esm.inv_cost = pyo.Param(esm.gen_types, initialize={"pv":1201, "coal":502, "gas":203})
    esm.efficiency = pyo.Param(esm.gen_types, esm.hour, initialize=eff_df_long.loc[:,t_start:t_end,:].to_dict()["value"])


    esm.generation = pyo.Var(esm.gen_types, esm.hour, within=pyo.NonNegativeReals)
    # if invest:
    esm.capacity = pyo.Var(esm.gen_types, within=pyo.NonNegativeReals)
    # else:
    # esm.capacity = pyo.Param(esm.gen_types, initialize={'pv': 584261.407579273, 'coal': 54650.0, 'gas': 1018.0})

    if demand_response:
        esm.demand_shift_up = pyo.Var(esm.hour, within=pyo.NonNegativeReals, bounds=(0,3000))
        esm.demand_shift_down = pyo.Var(esm.hour, within=pyo.NonNegativeReals, bounds=(0,3000))

    esm.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    return esm


## constraints

In [10]:
def add_constraints(
    esm,
    demand_response=False,
    capacity_limits: dict = None,
    gen_share_limits: dict = None,
):
    # energy balance
    def balance(m, h):
        if demand_response:
            return (
                m.demand[h]
                <= sum(m.generation[gt, h] for gt in m.gen_types)
                - m.demand_shift_up[h]
                + m.demand_shift_down[h]
            )
        else:
            return m.demand[h] <= sum(m.generation[gt, h] for gt in m.gen_types)

    if hasattr(esm, "balance_constraint"):
        esm.del_component("balance_constraint")
    esm.balance_constraint = pyo.Constraint(esm.hour, rule=balance)

    if demand_response:

        def demand_response_constraint(m, h):
            # print(h,h+1 in m.hour)
            return sum(
                m.demand_shift_up[h - i] for i in range(4) if h - i in m.hour
            ) == sum(m.demand_shift_down[h + i] for i in range(4) if h + i in m.hour)

        esm.dr_constraint = pyo.Constraint(esm.hour, rule=demand_response_constraint)

    # capacity constraint
    def cap_const(m, h, gt):
        return m.generation[gt, h] <= m.capacity[gt] * m.efficiency[gt, h]

    esm.capacity_constraint = pyo.Constraint(esm.hour, esm.gen_types, rule=cap_const)

    if capacity_limits:
        # print("Warning! Very simple implementation: pv < 30k")
        def cap_limit(m, gt, lim):
            return m.capacity[gt] <= lim

        esm.cap_limit = pyo.Constraint(
            list(capacity_limits.keys()), list(capacity_limits.values()), rule=cap_limit
        )

    if gen_share_limits:

        def gen_share_limit(m, gt, limit):
            return (
                sum(m.generation[gt, h] for h in m.hour)
                / sum(m.demand[h] for h in m.hour)
                <= limit
            )

        esm.gen_share_limit = pyo.Constraint(
            list(gen_share_limits.keys()), list(gen_share_limits.values()), rule=gen_share_limit
        )

    def cost(
        m,
    ):
        return sum(
            m.generation[gt, h] * m.fuel_cost[gt] for gt in m.gen_types for h in m.hour
        ) + sum(m.capacity[gt] * m.inv_cost[gt] for gt in m.gen_types)

    if hasattr(esm, "objective"):
        esm.del_component("objective")
    esm.objective = pyo.Objective(rule=cost)

    return esm

In [11]:
esm = define_model(demand_response=False)
esm = add_constraints(esm)

In [12]:
opt = pyo.SolverFactory("gurobi")
result = opt.solve(esm)
opt_vars = model_vars_as_dfs(esm)

dual_df = get_shadow_prices(esm)


In [13]:
# opt_vars
# dual_df

In [14]:
pyo.value(esm.objective)
# 15514472240.481754
#     1015756.107998 (sum of marginals)
# so with all demand + 1 should be:
# ontario_demand["Ontario Demand"] += 1
# 15515487996... 
# and it is!! (for coal_limit=False )
# 15515487996.589754

16033003201.809992

In [15]:
# dual_df = dual_dfs["balance_constraint"]
price_duration = dual_df.sort_values(by="dual", ascending=False).reset_index(names=["old_idx"])
fig = px.line(price_duration.reset_index(), x="index", y="dual", hover_data=["old_idx"])
fig.update_layout(width=600, height=400, title="load duration curve")

In [16]:
# if demand_response:
#     dr_df = opt_vars["demand_shift_down"].rename({"value":"down"}, axis=1)
#     dr_df["up"] = opt_vars["demand_shift_up"]["value"]
#     # dr_df.set_index("hour").plot()


# dual_df["old_demand"] = ontario_demand.loc[t_start:t_end,"Ontario Demand"]
# dual_df["post_dr_demand"] = ontario_demand.loc[t_start:t_end,"Ontario Demand"]

# if demand_response:
#     dual_df["post_dr_demand"] = dual_df["old_demand"] + dr_df["up"].values - dr_df["down"].values
#     dual_df["dr_shift"] = dr_df["up"].values - dr_df["down"].values

# # dual_df.melt(ignore_index=False)
# fig = px.line(dual_df.melt(ignore_index=False).reset_index(),x="index",y="value", facet_row="variable" )
# fig.update_yaxes(matches=None)
# # ontario_demand.loc[t_start:t_end,"Ontario Demand"].plot()

In [17]:
price_duration["dual"].sum()
# 1015756.1079984119
# however, the marginals don't change for
# ontario_demand["Ontario Demand"] += 1
# 1015756.1079984119

# nor do they change for 
# ontario_demand["Ontario Demand"] *= 2
# 1015756.1079984119

# but it does change for random demand additions:
# 1014357.3627871022 # ... but decreases?
# so shape shifting, is the only way to change marginals

# for demand.sum = 1
# 1049787.9184475108

1049791.8377495217

In [18]:
# price_duration["dual"].mean()

# 119.83880347574325 # <- with invest
# 119.80342465753425 # <- without invest


In [19]:
# (dual_dfs["balance_constraint"]["dual"] * dual_df["post_dr_demand"]).sum() / dual_df["post_dr_demand"].sum()
# 114.62626668476345 <- without invest
# 114.68167861367824 <- with invest

# 166.302 with dr=3000, and 4 hours each direction and pv_cap=30k
# 130.542 with dr=3000, and 4 hours each direction and no pv_cap!
# 136.342 with no dr and no pv_cap
# 169.195 with no dr and pv_cap=30k

In [20]:
# pyo.value(esm.objective) / dual_df["post_dr_demand"].sum()

# note that the introduction of a pv capacity limit alters sth, such that S_c/sum(D) != weighted marginal
# 141.049 with dr=3000, and 4 hours each direction and pv_cap=30k
# 130.542 with dr=3000, and 4 hours each direction and no pv_cap!
# 136.342 with no dr and no pv_cap
# 145.631 with no dr and pv_cap=30k

In [21]:

fig = px.area(opt_vars["generation"], x="hour", y="value", color="gen_types")
fig.update_traces(line=dict(width=0))

fig.update_layout(width=600, height=400)

In [22]:
gen_type_sum = opt_vars["generation"].pivot(columns="gen_types",index="hour",values="value").sum()
gen_type_sum/ gen_type_sum.sum()

gen_types
coal    0.528798
gas     0.000012
pv      0.471190
dtype: float64

In [23]:
opt_vars["capacity"]

,gen_types,value
0,pv,584261.407579
1,coal,54650.000000
2,gas,1018.000000


In [24]:

def get_model_demand(esm):
    hours = esm.demand.keys()
    demand = esm.demand.values()
    demand_ts = pd.Series(demand, index=hours)
    return demand_ts
# get_model_demand(esm).index[-1]

## Definition of the household DR esm

In [25]:
def define_hh_esm(
    demand_ts: pd.Series, price_ts: pd.Series, t_dr_shift=4, max_dr_demand_share=0.3
):
    hh_esm = pyo.ConcreteModel("Household electricity model")

    t_start = demand_ts.index[0]
    t_end = demand_ts.index[-1]

    hh_esm.hour = pyo.RangeSet(t_start, t_end)

    hh_esm.demand = pyo.Param(hh_esm.hour, initialize=demand_ts.to_dict())
    hh_esm.price = pyo.Param(hh_esm.hour, initialize=price_ts.to_dict())
    hh_esm.dr_up = pyo.Var(hh_esm.hour, within=pyo.NonNegativeReals)
    hh_esm.dr_down = pyo.Var(hh_esm.hour, within=pyo.NonNegativeReals)

    def demand_response_constraint(m, h):
        # print(h,h+1 in m.hour)
        return sum(m.dr_up[h - i] for i in range(t_dr_shift) if h - i in m.hour) == sum(
            m.dr_down[h + i] for i in range(t_dr_shift) if h + i in m.hour
        )

    hh_esm.dr_constraint = pyo.Constraint(hh_esm.hour, rule=demand_response_constraint)

    def max_dr(m, h, share):
        return m.dr_down[h] + m.dr_up[h] <= m.demand[h] * share
    hh_esm.dr_limit = pyo.Constraint(hh_esm.hour, [max_dr_demand_share], rule=max_dr)

    def dr_ramp_up(m, h, max_ramp=0.05):
        if h+1 in m.hour:
            return m.dr_up[h+1] - m.dr_up[h] <= m.demand[h] * max_dr_demand_share * max_ramp
        else:
            max_ramp = m.demand[h] * max_dr_demand_share
            return m.dr_up[h] <= max_ramp
    hh_esm.dr_ramp_up = pyo.Constraint(hh_esm.hour, rule=dr_ramp_up)

    def dr_ramp_down(m, h, max_ramp=0.05):
        if h+1 in m.hour:
            return m.dr_down[h+1] - m.dr_down[h]<= m.demand[h] * max_dr_demand_share * max_ramp
        else:
            max_ramp = m.demand[h] * max_dr_demand_share
            return m.dr_down[h] <= max_ramp
    hh_esm.dr_ramp_down = pyo.Constraint(hh_esm.hour, rule=dr_ramp_down)


    def objective(m):
        return sum(
            (m.demand[h] + m.dr_up[h] - m.dr_down[h]) * m.price[h] for h in m.hour
        )

    hh_esm.obj = pyo.Objective(rule=objective)

    return hh_esm


hh_esm = define_hh_esm(get_model_demand(esm), dual_df["dual"])

In [26]:
hh_result = opt.solve(hh_esm)

In [27]:
def get_effective_dr(hh_esm):
    hh_opt_vars = model_vars_as_dfs(hh_esm)
    dr_df = pd.DataFrame()
    dr_df["up"] = hh_opt_vars["dr_up"]["value"]
    dr_df["down"] = hh_opt_vars["dr_down"]["value"]
    dr_df["sum"] = dr_df["up"] - dr_df["down"]
    dr_df.index = hh_opt_vars["dr_down"]["hour"]
    return dr_df["sum"]
get_effective_dr(hh_esm).describe()

count    8.760000e+03
mean     2.180296e-15
std      9.559435e+02
min     -3.434625e+03
25%     -1.818413e+02
50%      2.667975e+02
75%      7.003050e+02
max      9.730650e+02
Name: sum, dtype: float64

# Couple both models

In [32]:
t_start = 3000
t_end = t_start + 12*24

initial_demand = ontario_demand["Ontario Demand"].loc[t_start:t_end]#.reset_index(drop=True)
demand_ts = get_model_demand(esm).loc[t_start:t_end]
dr_ts = pd.Series([0]*len(demand_ts), index=demand_ts.index).loc[t_start:t_end]#.reset_index(drop=True)

all_prices = []
all_drs = []
all_generation = []
all_demands = [demand_ts]
all_capacities = []

n_iterations = 5
for i in range(n_iterations):
    print("starting iteration",i)
    
    # define & solve model
    esm = define_model(demand_response=False, demand_ts=initial_demand+dr_ts, t_start=t_start, t_end=t_end)
    esm = add_constraints(esm, )#capacity_limits={"pv":30000, "coal":40000})
    esm_result = opt.solve(esm)
    esm_vars = model_vars_as_dfs(esm)

    # extract prices
    prices = get_shadow_prices(esm)["dual"]
    all_prices.append(prices)
    all_generation.append(esm_vars["generation"])

    # define and solve hh_esm
    hh_esm = define_hh_esm(demand_ts, prices)
    hh_result = opt.solve(hh_esm)
    
    # get dr and add to demand
    dr_ts = get_effective_dr(hh_esm)
    all_drs.append(dr_ts)

    all_demands.append(initial_demand+dr_ts)
    all_capacities.append(esm_vars["capacity"])
    


starting iteration 0
starting iteration 1
starting iteration 2
starting iteration 3
starting iteration 4


In [33]:
demands = pd.concat(all_demands, axis=1)
dr_df = pd.concat(all_drs, axis=1)
dr_df.columns = range(n_iterations)
dr_df = dr_df.melt(var_name="i", value_name="sum", ignore_index=False).reset_index()
dr_df

,hour,i,sum
0,3000,0,1.839600e+02
1,3001,0,-1.563194e-12
2,3002,0,1.879200e+02
3,3003,0,1.879200e+02
4,3004,0,4.547474e-12
...,...,...,...
1440,3284,4,6.121500e+02
1441,3285,4,5.899050e+02
1442,3286,4,-1.429905e+03
1443,3287,4,-1.627065e+03


In [34]:
for p, dem in zip(all_prices, all_demands):
    print((p* dem).sum()/dem.sum())

136.34263122975725
133.25416388422082
133.20194579826352
133.3383540807199
133.21944163438496


In [35]:
all_capacities

[  gen_types         value
 0        pv  63742.621477
 1      coal  41463.358907
 2       gas   1001.847230,
   gen_types         value
 0        pv  65625.475200
 1      coal  41826.960526
 2       gas   1013.240000,
   gen_types         value
 0        pv  66362.827880
 1      coal  41378.947368
 2       gas   1066.000000,
   gen_types         value
 0        pv  65625.475200
 1      coal  42316.881579
 2       gas   1416.370000,
   gen_types         value
 0        pv  65609.133732
 1      coal  41781.578947
 2       gas    760.000000]

In [36]:

px.line(dr_df, x="hour", y="sum", color="i")

In [37]:
px.line(all_drs[0].cumsum())

In [38]:
all_drs[0].sum()

4.547473508864641e-12

In [39]:
prices_df = pd.concat(all_prices, axis=1)
prices_df.columns = range(n_iterations)
prices_df = prices_df.melt(var_name="i", value_name="dual", ignore_index=False).reset_index()
prices_df

,index,i,dual
0,3000,0,207.000
1,3001,0,207.000
2,3002,0,207.000
3,3003,0,207.000
4,3004,0,207.000
...,...,...,...
1440,3284,4,0.001
1441,3285,4,0.001
1442,3286,4,0.001
1443,3287,4,207.000


In [40]:
px.line(prices_df, x="index", y="dual", color="i")

In [41]:
gen_df = pd.DataFrame()
for i, g_df in enumerate(all_generation):
    g_df["i"] = i
    gen_df = pd.concat([gen_df, g_df])

fig = px.area(gen_df, x="hour",y="value", color="gen_types", facet_row="i")

fig.update_layout(height=800)

In [42]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=len(all_drs), cols=1)
demand_df = demand_ts.to_frame(name="demand_0")
initial_demand_trace = go.Scatter(x=demand_ts.index, y=demand_ts, showlegend=False, line=dict(color="blue"))
for i,dr in enumerate(all_drs):
    # demand_df[f"post_dr_{i+1}"] = demand_df["demand_0"] + dr
    fig.add_trace(
        initial_demand_trace, row=i+1, col=1
    )
    fig.add_trace(
        go.Scatter(x=dr.index, y=demand_ts+dr, line=dict(dash="dash")), row=i+1, col=1
    )

fig.update_xaxes(matches="x")
fig.update_yaxes(matches="y")
fig.update_layout(height=800)

In [43]:
px.bar(gen_df.groupby(["i","gen_types"]).sum().reset_index(), x="i",y="value",color="gen_types")